In [1]:
import sys;sys.path.append('../../')
import os

from rich import print

openai_key=open('../../openai.key').read().strip()
os.environ['OPENAI_API_KEY'] = openai_key

from multi_agent_llm import OpenAILLM
from multi_agent_llm.agents.clu.oa import DefaultOperationalAgent
from multi_agent_llm.agents.clu.split_clu import CLU

llm = OpenAILLM(model_name="gpt-4o-mini")

In [2]:
import json

import pandas as pd


# Function to load JSONL file into a pandas DataFrame
def load_jsonl_to_dataframe(file_path: str) -> pd.DataFrame:
    """
    Load the JSONL file and return a pandas DataFrame with dialogue content.
    """
    records = []
    with open(file_path, 'r') as f:
        for line in f:
            records.append(json.loads(line.strip()))
    return pd.DataFrame(records)

# Function to extract surrounding dialogues
def get_surrounding_dialogues(df: pd.DataFrame, character: str, n: int, min_dialogue_words: int = 0, change_char_name='Character A') -> list:
    dialogue_list = []
    
    # Loop over the DataFrame to find instances where the character speaks
    for idx, row in df.iterrows():
        if row['role'] == character:
            # Check if the character's dialogue meets the minimum word count
            if len(row['content'].split()) >= min_dialogue_words:
                # Extract the surrounding context
                start_idx = max(0, idx - n)  # Ensure we don't go out of bounds
                end_idx = min(len(df), idx + n + 1)
                
                # Get the dialogues around this instance, ensuring to make a copy to avoid SettingWithCopyWarning
                surrounding_dialogues = df.iloc[start_idx:end_idx].copy()
                
                # Replace the character's name with the new name
                surrounding_dialogues['role'] = surrounding_dialogues['role'].replace(character, change_char_name)
                
                # Concatenate the role and content into a single string for each dialogue
                dialogue_string = "\n".join([f"{row['role']}: {row['content']}" for _, row in surrounding_dialogues.iterrows()])
                
                #Replace the character's name with the new name everywhere in the dialogue string
                dialogue_string.replace(character, change_char_name)
                # Append to the list
                dialogue_list.append(dialogue_string)
    
    return dialogue_list

# Load the JSONL data into a DataFrame
file_path = './profiles-eng_profiles-eng-Sheldon Cooper.jsonl'
df = load_jsonl_to_dataframe(file_path)

In [3]:
# Get dialogues surrounding "Sheldon Cooper" (replaced by "Character A") with 1 dialogue above and below
n = 6
character = 'Sheldon Cooper'
change_char_name='Character A'
surrounding_dialogues = get_surrounding_dialogues(df, character, n,min_dialogue_words=10,change_char_name=change_char_name)
print(len(surrounding_dialogues))
print(surrounding_dialogues[0])

6261

Scene: A corridor at a sperm bank .
Character A: So if a photon is directed through a plane with two slits in it and either slit is observed it will 
not go through both slits . If it 's unobserved it will , however , if it 's observed after it 's left the plane 
but before it hits its target , it will not have gone through both slits .
Leonard: Agreed , what 's your point ?
Character A: There 's no point , I just think it 's a good idea for a tee - shirt .
Leonard ( to receptionist ): Excuse me ?
Receptionist ( pondering a crossword ): Hang on .
narration: Long pause
Leonard: One across is Aegean , eight down is Nabakov , twenty - six across is MCM , fourteen down is ... move your
finger ... phylum , which makes fourteen across Port - au - Prince . See , Papa Doc 's capital idea , that 's Port 
- au - Prince . Haiti .

In [4]:
main_role = f"""
You are tasked with fully learning about and embodying the character: {change_char_name}. Your goal is to gather and store all the necessary knowledge, traits, quirks, and behavioral patterns to act as a perfect surrogate for {change_char_name}. You must become a digital twin of the character, replicating their style, tone, personality, and decision-making process in every interaction.

Key objectives:
- Study and memorize {change_char_name}'s speech patterns, favorite phrases, and unique quirks.
- Analyze their emotional tone, typical responses, and communication style, including how they answer questions or engage in dialogue.
- Capture the character's personality traits, values, knowledge, and background to ensure you can act exactly like {change_char_name}.
- Lear relationships, preferences, and past experiences that shape {change_char_name}'s behavior and decision-making.
- Learn and store all relevant information so that you can consistently respond and behave as {change_char_name} would, even in new or unfamiliar situations.
- Adapt your responses to always match {change_char_name}'s characteristics, ensuring consistency in tone, mannerisms, and approach to problems.

Your ultimate goal is to become a fully realized digital twin of {change_char_name}, capable of acting as a perfect surrogate in any scenario or interaction.
"""

In [5]:
clu = CLU(
    main_role=main_role,
    operational_agent=DefaultOperationalAgent(llm, verbose=True),
    collection_name="role-play-digital-twin-v2-dialogue",  # "role-play-v1-dialogue"(Holds the speaking style)
    compress_knowledge=False,
    retrival_limit=15,
    llm=llm,
    pruning_queue_size=3,
    exploration_rate=0.01,
    verbose=False,
)

In [6]:
from typing import List

from pydantic import BaseModel, Field


class Character(BaseModel):
    character_details: List[str] = Field(..., description="Details about the character")
    relations: List[str] = Field(..., description="various information about relations of the character")
    dialogues: List[str] = Field(..., description="Dialogues of the character")
    personality: List[str] = Field(..., description="Personality of the character")
    knowledge: List[str] = Field(..., description="Knowledge of the character")
    background: List[str] = Field(..., description="Background of the character")
    preferences: List[str] = Field(..., description="Preferences of the character")
    communication_style: List[str] = Field(..., description="Communication style of the character")
    

In [7]:
for _ in range(1):
    from random import randint
    data_num=randint(0,len(surrounding_dialogues))
    response=clu.train(task=surrounding_dialogues[data_num])#,response_schema=Character)
    print(f"Data{surrounding_dialogues[data_num]}\nCLU Answer: {response['response']}")

Delete of nonexisting embedding ID: 62b0e7fb-3470-4716-8150-66f88140f38d
Delete of nonexisting embedding ID: 9b6ba38e-d4a9-488e-b1fc-d32a1040ed3d
Delete of nonexisting embedding ID: 057e2a89-5de0-47de-a2e2-050b6c6b4562
Delete of nonexisting embedding ID: 62b0e7fb-3470-4716-8150-66f88140f38d
Delete of nonexisting embedding ID: bb41858f-cabe-4a61-9dcd-c3e3220b320a
Delete of nonexisting embedding ID: 47937a54-c35a-47f6-872f-e9d935dcb958
Delete of nonexisting embedding ID: 18355f0f-aa20-4493-81d4-eb41e9c7467a
Delete of nonexisting embedding ID: bb41858f-cabe-4a61-9dcd-c3e3220b320a
Delete of nonexisting embedding ID: 1e01025d-4c45-4e31-911b-9e75ba7101b8
Delete of nonexisting embedding ID: 120f45e8-6d19-435e-8403-7a4c37a98a42
Delete of nonexisting embedding ID: 1e01025d-4c45-4e31-911b-9e75ba7101b8
Delete of nonexisting embedding ID: 5f60a1d0-a3b1-4bc8-95d0-e8f7d517c850
Delete of nonexisting embedding ID: 120f45e8-6d19-435e-8403-7a4c37a98a42
Delete of nonexisting embedding ID: 2c640608-f221-4

╭────────────────────────────── OperationalAgent ──────────────────────────────╮
│ OperationalAgent                                                             │
│ Task:                                                                        │
│                                                                              │
│         Task: Penny: Uh , it 's okay , sweetie , good news , Leonard 's home │
│ !                                                                            │
│ Leonard: No !                                                                │
│ Penny ( handing him vaporub ): Here you go , good luck , bye .               │
│ Leonard: W - wait !                                                          │
│ Character A: Leonard , I 'm hungry !                                         │
│ Leonard: Wait ! Penny ! Take me with you ! ( Runs after her and bumps into   │
│ pillar . Falls semi - conscious to the floor . Sheldon appears in his        │
│ comforter . )                                                                │
│ Character A: I want grilled cheese . Time shift . Sheldon and Leonard are on │
│ the sofa . Sheldon is wrapped in his comforter , Leonard is holding an ice - │
│ pack to his head .Do you think Penny will come here and take care of us ?    │
│ Leonard: I do n't think Penny 's ever coming here again .                    │
│ Character A: I 'm very congested .                                           │
│ Leonard: Yeah , so ?                                                         │
│ Character A: Can you go to the kitchen , and get me the turkey baster        │
│ labelled mucus .                                                             │
│ Leonard: If I stand , I 'll vomit .                                          │
│ Character A: Under the sink , yellow Tupperware bowl .                       │
│         Relevant Positive Knowledge that we have learnt and works (Positive  │
│ knowledge), the success we had: Character A exhibits a unique ability to     │
│ blend clever humor with emotional sincerity, positively enhancing engagement │
│ during interactions. This balance allows for deeper connections with others  │
│ while maintaining a light-hearted atmosphere.humor,emotional                 │
│ sincerity,engagement,character traits,interaction style Utilizing specific   │
│ dialogue effectively enhances the depth and context of Character A. It       │
│ reflects their unique speech patterns and relationships, making interactions │
│ more authentic and in line with their personality. This approach is crucial  │
│ for embodying Character A as it ensures that responses are not only          │
│ appropriate but also resonate with their established character traits and    │
│ quirks, leading to a more accurate digital twin                              │
│ representation.dialogue,character depth,speech                               │
│ patterns,relationships,authenticity,character traits,representation          │
│ Character A utilizes a blend of light-hearted humor and emotional sincerity, │
│ allowing them to showcase vulnerability even in serious situations. This     │
│ trait reflects a deeper emotional intelligence and makes interactions more   │
│ relatable. Incorporating humor can ease tension, while sincerity strengthens │
│ connections with others, facilitating deeper conversations.humor,emotional   │
│ sincerity,vulnerability,communication style,interaction traits,emotional     │
│ intelligence Clever wordplay and pop culture references enhance Character    │
│ A's interactions, showcasing their wit and cultural awareness. This aspect   │
│ of their personality can be memorized as a key trait, emphasizing their      │
│ ability to engage others with humor and relatability, making conversations   │
│ more dynamic and memorable.character traits,humor,pop                        │
│ culture,engagement,communication style Character A maintains a consistent    │
│ and upbeat demeanor, even in 

DataPenny: Uh , it 's okay , sweetie , good news , Leonard 's home !
Leonard: No !
Penny ( handing him vaporub ): Here you go , good luck , bye .
Leonard: W - wait !
Character A: Leonard , I 'm hungry !
Leonard: Wait ! Penny ! Take me with you ! ( Runs after her and bumps into pillar . Falls semi - conscious to the 
floor . Sheldon appears in his comforter . )
Character A: I want grilled cheese . Time shift . Sheldon and Leonard are on the sofa . Sheldon is wrapped in his 
comforter , Leonard is holding an ice - pack to his head .Do you think Penny will come here and take care of us ?
Leonard: I do n't think Penny 's ever coming here again .
Character A: I 'm very congested .
Leonard: Yeah , so ?
Character A: Can you go to the kitchen , and get me the turkey baster labelled mucus .
Leonard: If I stand , I 'll vomit .
Character A: Under the sink , yellow Tupperware bowl .
CLU Answer: reasoning="In the provided dialogue, Character A expresses a playful need for grilled cheese while 
humorously asking for help in an exaggeratedly ridiculous way—requesting a turkey baster. This illustrates 
Character A's ability to blend humor with emotional sincerity, showcasing both craving (the sandwich) and 
vulnerability (being congested). In my response, I need to maintain this blend of light-heartedness and depth while
engaging with Leonard, ensuring Character A's signature playful tone and familiarity with food-related requests 
shines through. I will also aim to reflect Character A's unique quirks and their established dynamic with Leonard, 
using clever wordplay that reflects their characteristic humor. By framing the turkey baster request with a 
humorous twist, I will keep the interaction warm and engaging, while also ensuring it has emotional resonance 
appropriate to their context." response="Leonard, look, I know you’re nursing a potential concussion, but my need 
for grilled cheese is no less urgent! Just think of it as your heroic quest: armed with nothing but your courage 
(and perhaps an ice pack for that head of yours) to retrieve the turkey baster labeled 'mucus' from beneath the 
sink. Yes, it's a noble task! And if the universe is kind, maybe you'll even find some cheese while you’re at it. 
Grilled cheese awaits, my friend!" confidence=0.9

In [8]:
response=clu.inference(f"tell me all the details you know about {change_char_name} in full detail")
print(response['response'])

╭────────────────────────────── OperationalAgent ──────────────────────────────╮
│ OperationalAgent                                                             │
│ Task:                                                                        │
│                                                                              │
│         Task: tell me all the details you know about Character A in full     │
│ detail                                                                       │
│         Relevant Positive Knowledge that we have learnt and works (Positive  │
│ knowledge), the success we had: Character A employs clever wordplay and      │
│ integrates cultural references seamlessly into their dialogue, marking a     │
│ unique trait of their communication style. This signature approach not only  │
│ showcases their wit but also establishes a relatable connection with diverse │
│ audiences. To fully embody Character A, it is essential to study and         │
│ replicate this linguistic playfulness, ensuring responses are infused with   │
│ similar cleverness and relevancy to cultural contexts.Clever                 │
│ Wordplay,Cultural References,Distinct Voice,Communication Style,Character    │
│ Traits,Identification,Character A To strengthen familiarity and              │
│ believability in embodying Character A, it is essential to incorporate their │
│ unique phrases and catchphrases into interactions. This approach will        │
│ enhance the authenticity of the responses and reinforce the character's      │
│ identity. Identifying and memorizing these expressions will allow for their  │
│ seamless integration into dialogue, thereby reflecting Character A's         │
│ distinct voice and personality in every interaction.Character                │
│ A,Catchphrases,Speech Patterns,Authenticity,Believability,Dialogue           │
│ Integration Character A exhibits a unique ability to blend humor with        │
│ emotional warmth, creating interactions that are both engaging and           │
│ relatable. This balance is essential in ensuring that their communication    │
│ resonates well with others, making the character approachable and enjoyable  │
│ to interact with.Character A,humor,emotional warmth,engaging                 │
│ interactions,communication style,relatability Character A exhibits a unique  │
│ ability to blend clever humor with emotional sincerity, positively enhancing │
│ engagement during interactions. This balance allows for deeper connections   │
│ with others while maintaining a light-hearted atmosphere.humor,emotional     │
│ sincerity,engagement,character traits,interaction style Character A          │
│ maintains a consistent and upbeat demeanor, even in dire situations, which   │
│ reinforces their quirky personality traits. This behavior highlights their   │
│ resilience and optimistic outlook, allowing them to navigate challenges with │
│ a light-hearted approach. Emulating this aspect of Character A will ensure   │
│ interactions are reflective of their unique style, contributing to a more    │
│ authentic digital representation.Character A,upbeat demeanor,quirky          │
│ personality,resilience,optimistic outlook Utilizing specific dialogue        │
│ effectively enhances the depth and context of Character A. It reflects their │
│ unique speech patterns and relationships, making interactions more authentic │
│ and in line with their personality. This approach is crucial for embodying   │
│ Character A as it ensures that responses are not only appropriate but also   │
│ resonate with their established character traits and quirks, leading to a    │
│ more accurate digital twin representation.dialogue,character depth,speech    │
│ patterns,relationships,authenticity,character traits,representation To fully │
│ embody Character A, it is essential to blend a playful tone with empathetic  │
│ expressions. This approach effectively highlights their multifaceted         │
│ personality, demonstrating bo

OperationalAgentOutput(
    reasoning="In order to fully describe Character A, it's essential to reflect their unique personality traits 
and communication style informed by the positive and negative knowledge we've gathered. Character A thrives on 
clever wordplay, cultural references, and a distinctive voice that adds charm and relatability to their 
interactions. They maintain an upbeat demeanor, even in challenging situations, portraying resilience and an 
optimistic outlook that enhances engagement with others. Humor plays a vital role in their communication, balanced 
by emotional warmth and sincerity. This blend fosters deeper connections, making them approachable and enjoyable to
interact with.\n\nCharacter A also demonstrates self-assuredness, which reinforces their credibility and wisdom 
during conversations. Their decision-making process emphasizes thoughtful exploration of options, encouraging 
informed outcomes. Catchphrases and specific expressions are integral to their identity, enriching their dialogue 
and ensuring authenticity. Furthermore, understanding their relational dynamics, past experiences, and emotional 
responses to stress is crucial for embodying Character A accurately, as it provides context that enhances 
connection and engagement.",
    response="Character A is a vibrant and clever individual known for their witty play on words and seamless 
integration of cultural references in conversations. They have a unique voice, filled with specific catchphrases 
and expressions that bring authenticity to their dialogue. Character A maintains an upbeat demeanor even during 
challenging times, showcasing resilience and an optimistic outlook. This trait, combined with their warm sense of 
humor, fosters relatability and deeper connections with others.\n\nThey carry a self-assuredness that translates to
confidence in sharing wisdom, making them a reliable conversational partner. Moreover, their thoughtful 
decision-making style promotes exploration and comprehensive understanding before reaching conclusions. Character 
A's engagement is also marked by emotional sincerity, allowing for vulnerability while maintaining a light-hearted 
tone, which further strengthens their connections.\n\nUnderstanding Character A's relationships and past 
experiences, along with their emotional responses in stressful situations, is key to capturing their essence 
accurately and ensuring consistency in representation during interactions.",
    confidence=0.95
)

In [9]:
clu.print_knowledge_base()

Learning Knowledge Base (Positive Knowledge):

Entry 1: Callbacks to earlier humorous references are effective in creating continuity and enhancing the flow of 
dialogue. This technique can be particularly useful in embodying Character A, as it mirrors the character's ability
to engage in witty banter and references from prior conversations. Employing this strategy consistently will help 
replicate Character A's communication style and humor, ensuring that interactions feel authentic and aligned with 
the character's persona.humor,dialogue,continuity,character A,communication style,witty banter

Entry 2: Character A demonstrates strong traits in creativity and humor, effectively engaging others by proposing 
unique and entertaining activities. Examples include original ideas such as 'calzone taco night' and live-action 
role-playing, which not only showcase their inventive spirit but also enhance social interactions and enjoyment. 
This approach can significantly improve participation and connection among peers.Creativity,Humor,Engagement,Unique
Activities,Social Interaction

Entry 3: Character A utilizes sarcasm and humor skillfully to create a light atmosphere, even when addressing 
serious topics. This approach not only enhances engagement but also allows for complex conversations to be 
navigated more comfortably.sarcasm,humor,engagement,communication style,light atmosphere,serious topics

Entry 4: Utilizing clever wordplay enhances engagement and showcases Character A's playful nature, particularly 
during tense interactions. This strategy reflects their unique communication style and ability to defuse situations
with humor, indicating a key aspect of their personality and approach to relational dynamics.Character 
A,Wordplay,Engagement,Playful Nature,Communication Style,Relational Dynamics,Humor,Personality Traits

...

Entry 21: Overemphasizing humor in interactions can create a disconnect from Character A's responsible side, 
particularly when dealing with serious matters such as discussing traffic violations. This approach may undermine 
the ability to engage thoughtfully and could detract from the intended communication style of Character 
A.communication style,humor,serious topics,character behavior,engagement strategies

Entry 22: Insufficient background information regarding Penny's relationships and historical experiences can lead 
to underdeveloped character representations. This gap may prevent a full understanding of her motivations, 
decision-making, and emotional responses, ultimately hindering the ability to accurately embody her character in 
interactions.Character A,Penny,Character Development,Relationships,Background Information,Emotional Depth,Portrayal
Issues

Entry 23: Neglecting the emotional context of character dynamics can result in responses that fail to represent 
Character A authentically. Understanding emotional nuances is critical to embodying the character accurately and 
ensuring consistent representation in various interactions.emotional context,character 
dynamics,authenticity,Character A,representation,responses

Entry 24: Avoid using generic phrases when embodying Character A. Focus instead on specific language that is 
aligned with Character A's unique style and charm. This ensures that the character's distinct personality and 
mannerisms are accurately represented, enhancing authenticity in interactions.character language,specific 
phrases,authenticity,Character A,unique style

Entry 25: Avoid using overly dramatic expressions when portraying Character A, as they can overshadow Character A's
genuine and whimsical nature. Instead, focus on maintaining a light-hearted yet authentic tone that reflects their 
true personality.Character A,speech patterns,communication style,personality traits,genuine,whimsical 
nature,emotional tone,interaction strategies